In [116]:
#%reset -f

import csv
import os

import numpy as np
import matplotlib.pyplot as plt
from pyquaternion import Quaternion
%matplotlib tk

# load stuff
traj_name = 'lemniscate' # 'circle'

ref_csv_fn = 'data/ref_' + traj_name + '.csv'

ref_traj_fn = 'data/imu_simulator/ref/' + traj_name + '/trajectory.txt'
ref_vel_fn = 'data/imu_simulator/ref/' + traj_name + '/velocity.txt'
ref_imu_fn = 'data/imu_simulator/ref/' + traj_name + '/simulated_imu_meas.txt'

sim_traj_fn = 'data/imu_simulator/sim/' + traj_name + '/trajectory.txt'
sim_vel_fn = 'data/imu_simulator/sim/' + traj_name + '/velocity.txt'
sim_imu_fn = 'data/imu_simulator/sim/' + traj_name + '/simulated_imu_meas.txt'

with open(ref_csv_fn, 'r') as csvfile:
    ref_csv_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    ref_csv_array = np.array(list(ref_csv_reader))[1:]
    csv_ref = ref_csv_array.astype(float)
    
csv_traj = csv_ref[:, 0:8]
csv_vel = csv_ref[:, 8:11]
csv_gyro = csv_ref[:, 11:14]
    
ref_traj = np.loadtxt(ref_traj_fn)
ref_vel = np.loadtxt(ref_vel_fn)
ref_imu = np.loadtxt(ref_imu_fn)

sim_traj = np.loadtxt(sim_traj_fn)
sim_vel = np.loadtxt(sim_vel_fn)
sim_imu = np.loadtxt(sim_imu_fn)

In [117]:
# Make times start from 0.0
csv_ts = csv_traj[:,0] - csv_traj[0,0]
t_offset = 0.0 #-0.5
csv_ts -= t_offset

ref_ts = ref_traj[:,0] - ref_traj[0,0]

sim_ts = sim_traj[:,0] - sim_traj[0,0]
t_offset = 1.25
sim_ts -= t_offset
sim_imu[:,0] -= sim_imu[0,0]
sim_imu[:,0] -= t_offset

In [118]:
# Plot positions
plt.figure('XY traj')
plt.plot(ref_traj[:,1], ref_traj[:,2], label='reference')
plt.plot(sim_traj[:,1], sim_traj[:,2], label='sim')
plt.plot(csv_traj[:,1], csv_traj[:,2], 'r', label='csv', alpha=0.5)
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

plt.figure('XZ traj')
plt.plot(ref_traj[:,1], ref_traj[:,3], label='reference')
plt.plot(sim_traj[:,1], sim_traj[:,3], label='sim')
plt.plot(csv_traj[:,1], csv_traj[:,3], 'r', label='csv', alpha=0.5)
plt.xlabel('x')
plt.ylabel('z')
plt.legend()

plt.figure('YZ traj')
plt.plot(ref_traj[:,2], ref_traj[:,3], label='reference')
plt.plot(sim_traj[:,2], sim_traj[:,3], label='sim')
plt.plot(csv_traj[:,2], csv_traj[:,3], 'r', label='csv', alpha=0.5)
plt.xlabel('y')
plt.ylabel('z')
plt.legend()

In [119]:
# Plot pos - time
plt.figure('X time')
plt.plot(ref_ts, ref_traj[:,1], label='reference')
plt.plot(sim_ts, sim_traj[:,1], label='sim')
plt.plot(csv_ts, csv_traj[:,1], 'r', label='csv', alpha=0.5)
plt.xlabel('t')
plt.ylabel('x')
plt.legend()

plt.figure('Y time')
plt.plot(ref_ts, ref_traj[:,2], label='reference')
plt.plot(sim_ts, sim_traj[:,2], label='sim')
plt.plot(csv_ts, csv_traj[:,2], 'r', label='csv', alpha=0.5)
plt.xlabel('t')
plt.ylabel('y')
plt.legend()

plt.figure('Z time')
plt.plot(ref_ts, ref_traj[:,3], label='reference')
plt.plot(sim_ts, sim_traj[:,3], label='sim')
plt.plot(csv_ts, csv_traj[:,3], 'r', label='csv', alpha=0.5)
plt.xlabel('t')
plt.ylabel('z')
plt.legend()

In [120]:
# Plot velocity
plt.figure('X vel')
plt.plot(ref_ts, ref_vel[:,1], label='reference')
plt.plot(sim_ts, sim_vel[:,1], label='sim', alpha=0.7)
#plt.plot(csv_ts, csv_vel[:,1], 'r', label='csv', alpha=0.5)
plt.title('x vel')
plt.xlabel('t')
plt.ylabel('x')
plt.legend()

plt.figure('y vel')
plt.plot(ref_ts, ref_vel[:,2], label='reference')
plt.plot(sim_ts, sim_vel[:,2], label='sim', alpha=0.7)
#plt.plot(csv_ts, csv_vel[:,2], 'r', label='csv', alpha=0.5)
plt.title('y vel')
plt.xlabel('t')
plt.ylabel('y')
plt.legend()

plt.figure('z vel')
plt.plot(ref_ts, ref_vel[:,3], label='reference')
plt.plot(sim_ts, sim_vel[:,3], label='sim', alpha=0.7)
#plt.plot(csv_ts, csv_vel[:,3], 'r', label='csv', alpha=0.5)
plt.title('z vel')
plt.xlabel('t')
plt.ylabel('z')

plt.legend()

plt.show()

In [121]:
# Add rotor drag as in rotorS
# Eq. 2 in Furrer F., RotorS – A Modular Gazebo MAV Simulator
mass = 0.875
Ct = 1.56252e-06
Cd = 8.06428e-05
e_z = np.array([0., 0., 1.])
W_g = np.array([0.0, 0.0, 9.81])

n_samples = csv_traj.shape[0]
Fd = [] # Fdi = [Fd_rot1 (dim = (3,)) , Fd_rot2, Fd_rot3, Fd_rot4]
for i, t in enumerate(csv_ref):
    vA_i = np.array([t[8], t[9], t[10]])
    u_i = np.array([t[20], t[21], t[22], t[23]])
    
    # get rotor speeds
    w_i = np.sqrt(u_i * (1.0/Ct))
    
    W_acc_i = np.array([t[14], t[15], t[16]]) + g_w
    cthrust = W_acc_i
    e_zB = cthrust * (1/np.linalg.norm(cthrust))
    
    vA_I_perp = vA_i - (vA_i.dot(e_zB)) * e_zB
    
    Fd_i = np.zeros((3,4))
    for j in range(4):
        Fd_i[:,j] = -1.0 * w_i[j] * Cd * vA_I_perp
        
    Fd.append(Fd_i)

# add drag to accels of spline fitted on the ref. traj.
acc_rotorS = ref_imu[:, [0,4,5,6]]
acc_rotorS2 = []

n_samples = np.minimum(n_samples, acc_rotorS.shape[0])
for i in range(n_samples):
    acc_rotorS[i, 1:] += np.sum(Fd[i], axis=1) * (1/mass)

In [122]:
plt.figure('Acc')
plt.subplot(311)
plt.plot(ref_imu[:,0], ref_imu[:,4], label='reference')
plt.plot(sim_imu[:,0], sim_imu[:,4], label='sim')
plt.plot(acc_rotorS[:,0], acc_rotorS[:,1], label ='reference + rotorS drag')
plt.title('x acc')
plt.xlabel('t')
plt.ylabel('x')
plt.legend()

plt.subplot(312)
plt.plot(ref_imu[:,0], ref_imu[:,5], label='reference')
plt.plot(sim_imu[:,0], sim_imu[:,5], label='sim')
plt.plot(acc_rotorS[:,0], acc_rotorS[:,2], label ='reference + rotorS drag')
plt.title('y acc')
plt.xlabel('t')
plt.ylabel('y')
plt.legend()

plt.subplot(313)
plt.plot(ref_imu[:,0], ref_imu[:,6], label='reference')
plt.plot(sim_imu[:,0], sim_imu[:,6], label='sim')
plt.plot(acc_rotorS[:,0], acc_rotorS[:,3], label ='reference + rotorS drag')
plt.title('z acc')
plt.xlabel('t')
plt.ylabel('z')
plt.legend()

In [123]:
plt.figure('Gyro')
plt.subplot(311)
plt.plot(ref_imu[:,0], ref_imu[:,1], label='reference')
plt.plot(sim_imu[:,0], sim_imu[:,1], label='sim', alpha=0.7)
plt.title('x gyro')
plt.xlabel('t')
plt.ylabel('x')
plt.legend()

plt.subplot(312)
plt.plot(ref_imu[:,0], ref_imu[:,2], label='reference')
plt.plot(sim_imu[:,0], sim_imu[:,2], label='sim', alpha=0.7)
plt.title('y gyro')
plt.xlabel('t')
plt.ylabel('y')
plt.legend()

plt.subplot(313)
plt.plot(ref_imu[:,0], ref_imu[:,3], label='reference')
plt.plot(sim_imu[:,0], sim_imu[:,3], label='sim', alpha=0.7)
plt.title('z gyro')
plt.xlabel('t')
plt.ylabel('z')
plt.legend()